<img src="escudo_utfsm.gif" style="float:right;height:100px">
<img src="IsotipoDIisocolor.png" style="float:left;height:100px">
<center>
    <h1> ILI285 - Computación Científica I / INF285 - Computación Científica</h1>
    <h1> Tarea 4: PageRank y GMRes </h1>
    <h2> César Quiroz Mansilla </h2>
    <h2> 201573578-6 </h2>
    <h2> cesar.quirozm@sansano.usm.cl </h2>
    <h3> [S]cientific [C]omputing [T]eam 2019</h3>
</center>



In [7]:
import numpy as np
import matplotlib.pyplot as plt
import random
import scipy as sp
from time import time
from ipywidgets import interact, IntSlider
#if not instaled: pip3 install networkx
import networkx as nx
from scipy.sparse.linalg import gmres

## PageRank

Para poder trabajar con el algoritmo, es necesario considerar inicialmente una matriz de adyacencia $A \in \mathbb{R}^{n \times n}$, con $n$ la cantidad de páginas web. Las entradas $a_{ij}$ de esta matriz tienen el valor 1 si la página $i$ tiene un enlace a la página $j$ y 0 en caso contrario. Notar que no necesariamente la matriz $A$ es simétrica, lo que denota que dos páginas web distintas podrían no enlazarse mutuamente. Además, considere que una misma página no se enlazará consigo misma, por lo que la matriz tendrá cero en su diagonal principal. 

Adicionalmente, podrían darse casos de que existan páginas que solo tienen links hacia ellas, pero no tienen links hacia otras páginas. En una representación de la matriz de adyacencia como grafo, se le conoce a estas páginas como nodos _sumideros_. Una consecuencia de estos casos podía ser que usuarios que llegan a esas páginas quedan retenidos porque no existen links a los cuales seguir navegando. Para evitar esta situación, se agregará la perturbación _rank-one_ a la matriz de adyacencia $A$.

$$
    \tilde{A} = A + \mathbf{a}\cdot\mathbf{1}^T,
$$

donde $\mathbf{a} \in \mathbb{R}^{n}$ es un vector con un $1$ en la componente que corresponde a los nodos sumideros  y $0$ en los otras componentes, el vector $\mathbf{1}$ corresponde al vector de unos en $\mathbb{R}^{n}$ y $^T$ es el operador transpuesta.

#### Dataset
Considere los datasets de los archivos `adjacency1.dat`, `adjacency2.dat`, `adjacency3.dat` y `adjacency4.dat`, que representan matrices de adyacencia tales que:
- Adjacency1 es una matriz de adyacencia de 100 páginas y alrededor de un 20% de elementos no nulos.
- Adjacency2 es una matriz de adyacencia de 100 páginas y alrededor de un 50% de elementos no nulos.
- Adjacency3 es una matriz de adyacencia de 100 páginas y alrededor de un 80% de elementos no nulos.
- Adjacency4 es una matriz de adyacencia de 1000 páginas y alrededor de un 5% de elementos no nulos.

Cada fila $i$ de un archivo dataset corresponde a una página de índice $i$, y todos los valores separados por espacios en dicha fila representan los índices de las páginas $j$ a las cuales $i$ apunta. En otras palabras, un archivo dataset registra los vértices del grafo de adyacencia.

Considere la siguiente función,`read_adjacency_matrix` , que obtiene la matriz de adyacencia a partir de los archivos anteriores.

In [12]:
'''
Input:
filename - (string) name of adjacency matrix file
Output: 
A - (n x n matrix) adjacency matrix
'''
def read_adjacency_matrix(file_path):
    adjacency_list = []
    with open(file_path, 'r') as f:
        for line in f:
            adjacency_list.append(np.array(list(map(int, line.split()))))
    n = len(adjacency_list)
    A = np.zeros((n,n))
    for i in range(n):
        A[i, adjacency_list[i]] = 1
    return A
A = read_adjacency_matrix("adjacency1.dat")
B = read_adjacency_matrix("adjacency2.dat")
C = read_adjacency_matrix("adjacency3.dat")
D = read_adjacency_matrix("adjacency4.dat")

## Sección 1 : Comparación de soluciones con GMRes y PALU

En esta sección se compararán las soluciones de PageRank obtenidas por medio de GMRes y PALU. Para esto, se considerarán los datasets de los archivos `adjacency1.dat`, `adjacency2.dat`, `adjacency3.dat` y `adjacency4.dat`, variaciones en el _damping factor_ $\alpha$ y el número de iteraciones $k$ de GMRes.

**1.** Construya el sistema lineal necesario para encontrar PageRank. Para ello desarrolle la función `build_linear_system`, que recibe una matriz de adyacencia $A$ y un _damping factor_ $\alpha$.

In [13]:
'''
Input:
A - (n x n matrix) adjacency matrix
alpha - (float) damping factor, takes values from 0 to 1
Output: 
A_hat - (n x n matrix) matrix of linear system
b_hat - (n vector) right hand side vector of linear system
'''
def build_linear_system(A, alpha):
    #matriz P 
    P = np.zeros((len(A),len(A)))
    P=A
    v1=0
    for x in P:
        v2=0
        s=sum(x)
        for i in x:
            if i==1:
                P[v1][v2]=1/s
            v2=v2+1
        v1=v1+1
    #matriz I
    I=np.zeros((len(A),len(A)))
    np.fill_diagonal(I,1)
    #Matriz v
    v=np.full((len(A), 1), 1/len(A))
    #b_hat
    b_hat=np.zeros((len(A),len(A)))
    b_hat=(1-alpha)*v
    #A_hat
    A_hat=np.zeros((len(A),len(A)))
    A_hat=I-(alpha*np.transpose(P))               
    return A_hat, b_hat


**2.** Considere el error $e_k = \|\mathbf{x^{k}_{G}}-\mathbf{x_{P}}\|_2$ una métrica de error que compara $\mathbf{x_{P}}$, la solución de PageRank obtenida por PALU, con $\mathbf{x^{k}_{G}}$ la solución de PageRank obtenida con $k$ iteraciones de GMRes. Construya un gráfico que muestre $e_k$ versus $k$ y utilice un widget para seleccionar un dataset y variar el valor del _damping factor_ $\alpha$. ¿Qué puede decir de la información mostrada en el gráfico? ¿Cómo afecta $\alpha$ en los resultados obtenidos?

In [14]:
###PALU
def solve_triangular(A, b, upper=True):
    n = b.shape[0]
    x = np.zeros_like(b)
    if upper==True:
        #perform back-substitution
        x[-1] = (1./A[-1,-1]) * b[-1]
        for i in range(n-2, -1, -1):
            x[i] = (1./A[i,i]) * (b[i] - np.sum(A[i,i+1:] * x[i+1:]))
    else:
        #perform forward-substitution
        x[0] = (1./A[0,0]) * b[0]
        for i in range(1,n):
            x[i] = (1./A[i,i]) * (b[i] - np.sum(A[i,:i] * x[:i]))
    return x

def palu_decomp(A):
    N,_ = A.shape
    P = np.identity(N)
    L = np.zeros((N,N))
    U = np.copy(A)
    #iterating through columns
    for j in range(N-1):
        #determine the new pivot
        p_index = np.argmax(np.abs(U[j:,j]))
        if p_index != 0:
            row_perm(P, j, j+p_index)
            row_perm(U, j, j+p_index)
            row_perm(L, j, j+p_index)
        #iterating through rows
        for i in range(j+1,N):
            L[i,j] = U[i,j]/U[j,j]
            U[i] -= L[i,j]*U[j]
    np.fill_diagonal(L,1)
    return P,L,U
def solve_palu(A, b):
    P,L,U = palu_decomp(A)
    #A.x = b -> P.A.x = P.b = b'
    b = np.dot(P,b)
    # L.c = b' with c = U.x
    c = solve_triangular(L, b, upper=False)
    x = solve_triangular(U, c)
    x=(1/sum(x))*x
    return x

##GMres
def GMRes(A, b, x0=np.array([0.0]), m=10, flag_display=False, threshold=1e-12):
    todas=np.zeros((m,len(b)))
    n = len(b)
    if len(x0)==1:
        x0=np.zeros(n)
    r0 = b - np.dot(A, x0)
    nr0=np.linalg.norm(r0)
    out_res=np.array(nr0)
    Q = np.zeros((n,n))
    H = np.zeros((n,n))
    Q[:,0] = r0 / nr0
    flag_break=False
    matris=0
    for k in np.arange(np.min((m,n))):
        y = np.dot(A, Q[:,k])
        if flag_display:
            print('||y||=',np.linalg.norm(y))
        for j in np.arange(k+1):
            H[j][k] = np.dot(Q[:,j], y)
            if flag_display:
                print('H[',j,'][',k,']=',H[j][k])
            y = y - np.dot(H[j][k],Q[:,j])
            if flag_display:
                print('||y||=',np.linalg.norm(y))
        # All but the last equation are treated equally. Why?
        if k+1<n:
            H[k+1][k] = np.linalg.norm(y)
            if flag_display:
                print('H[',k+1,'][',k,']=',H[k+1][k])
            if (np.abs(H[k+1][k]) > 1e-16):
                Q[:,k+1] = y/H[k+1][k]
            else:
                #print('flag_break has been activated')
                flag_break=True
            # Do you remember e_1? The canonical vector.
            e1 = np.zeros((k+1)+1)        
            e1[0]=1
            H_tilde=H[0:(k+1)+1,0:k+1]
        else:
            H_tilde=H[0:k+1,0:k+1]
        # Solving the 'SMALL' least square problem. 
        # This could be improved with Givens rotations!
        ck = np.linalg.lstsq(H_tilde, nr0*e1)[0] 
        if k+1<n:
            x = x0 + np.dot(Q[:,0:(k+1)], ck)
        else:
            x = x0 + np.dot(Q, ck)
        # Why is 'norm_small' equal to 'norm_full'?
        norm_small=np.linalg.norm(np.dot(H_tilde,ck)-nr0*e1)
        out_res = np.append(out_res,norm_small)
        if flag_display:
            norm_full=np.linalg.norm(b-np.dot(A,x))
            #print('..........||b-A\,x_k||=',norm_full)
            #print('..........||H_k\,c_k-nr0*e1||',norm_small);
        todas[matris]=x
        matris=matris+1
        #print(x)
        if flag_break:
            if flag_display: 
                print('EXIT: flag_break=True')
            break
        if norm_small<threshold:
            if flag_display:
                print('EXIT: norm_small<threshold')
            break
    return todas,out_res

In [15]:

def error(data,alpha,k):
    errores=[]
    if data==1:
        Ma,Vb=build_linear_system(A,alpha)
    elif data==2:
        Ma,Vb=build_linear_system(B,alpha)
    elif data==3:
        Ma,Vb=build_linear_system(C,alpha)
    else:
        Ma,Vb=build_linear_system(D,alpha)
    Px=solve_palu(Ma,Vb)   
    Vb=np.transpose(Vb)
    Gxk, _ = GMRes(Ma, Vb[0],m=k)
    for x in Gxk:
        con=0
        Gx=np.zeros((len(x),1))
        for i in x:
            Gx[con]=i
            con=con+1
        err=np.linalg.norm(Gx-Px)
        errores.append(err)
    lista = range(1,len(errores)+1)
    vx=list(lista)
    plt.scatter(vx, errores)
    plt.title("Plot k v/s error")
    plt.show()
    return False

interact(error,data=IntSlider(min=1,max=4,step=1,value=3),alpha=(0.1,1.0,0.01),k=IntSlider(min=1,max=100,step=1,value=10))

interactive(children=(IntSlider(value=3, description='data', max=4, min=1), FloatSlider(value=0.55, descriptio…

<function __main__.error(data, alpha, k)>

## Sección 2 : Tiempo de Ejecución

En esta sección se compararán los tiempos de ejecución de GMRes y PALU necesarios para resolver los sistemas de ecuaciones de PageRank. Para esto, se considerarán los datasets de los archivos `adjacency1.dat`, `adjacency2.dat`, `adjacency3.dat` y `adjacency4.dat`, variaciones en el _damping factor_ $\alpha$ y el número de iteraciones $k$ de GMRes.

**1.** Analice efecto de variar _damping factor_ $\alpha$ para encontrar las 10 primeras páginas entregadas por PageRank. Para ello utilice la función `get_damping_ranking` definida a continuación. 

In [16]:
def top(lista):
    list_top=[]
    n = lista
    idx = n.argsort()[-1]
    list_top.append(np.array((idx+1, n[idx])))
    idx2 = n.argsort()[-2]
    list_top.append(np.array((idx2+1, n[idx2])))
    idx3 = n.argsort()[-3]
    list_top.append(np.array((idx3+1, n[idx3])))
    idx4 = n.argsort()[-4]
    list_top.append(np.array((idx4+1, n[idx4])))
    idx5 = n.argsort()[-5]
    list_top.append(np.array((idx5+1, n[idx5])))
    idx6 = n.argsort()[-6]
    list_top.append(np.array((idx6+1, n[idx6])))
    idx7 = n.argsort()[-7]
    list_top.append(np.array((idx7+1, n[idx7])))
    idx8 = n.argsort()[-8]
    list_top.append(np.array((idx8+1, n[idx8])))
    idx9 = n.argsort()[-9]
    list_top.append(np.array((idx9+1, n[idx9])))
    idx10 = n.argsort()[-10]
    list_top.append(np.array((idx10+1, n[idx10])))    
    return list_top
        
'''
Input:
A - (n x n matrix) adjacency matrix
alpha - (float) damping factor, takes values from 0 to 1
k - number of iterations of GMRes until return a solution, use only if method is 'GMRes'
method - string that indicates the method used to solve the linear system. Take values 'PALU' or 'GMRes'
Output: 
ranking - list with 10 pages of ranking sorted by largest probability
'''
def get_damping_ranking(A, alpha, k, method='GMRes'):
    Ma,Vb=build_linear_system(A, alpha)
    Px=solve_palu(Ma,Vb)   
    Vb=np.transpose(Vb)       
    if "GMRes"== method:
        Gxk, _ = GMRes(Ma, Vb[0],m=k)
        Gx=np.zeros((len(Ma),1))
        Gx=Gxk[len(Gxk)-1]
        ranking=top(Gx)
        
    else:
        Px=np.transpose(Px)[0]
        ranking=top(Px)
    return ranking


get_damping_ranking(A,0.5,10)

C:\Users\cesar\Anaconda3\lib\site-packages\ipykernel_launcher.py:89: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


[array([2.40000000e+01, 1.25840274e-02]),
 array([3.70000000e+01, 1.19785908e-02]),
 array([7.40000000e+01, 1.19208987e-02]),
 array([6.00000000e+01, 1.16476491e-02]),
 array([5.00000000e+01, 1.15140201e-02]),
 array([9.60000000e+01, 1.15052893e-02]),
 array([3.40000000e+01, 1.13483387e-02]),
 array([1.        , 0.01130623]),
 array([6.80000000e+01, 1.13008833e-02]),
 array([8.90000000e+01, 1.12636799e-02])]

**2.** Construya un gráfico que muestre el tiempo de ejecución para determinar el ranking versus el factor de amortiguamiento $\alpha$. En el mismo gráfico debe mostrar los dos métodos utilizados (GMRes y PALU). Además, utilice un widget que permita seleccionar uno de los cuatro datasets mencionados y el número $k$ de iteraciones de GMRes. ¿Qué puede decir respecto de los resultados obtenidos en cada método al variar el valor de $\alpha$?

In [25]:
#metodo=1 GMRes
#metodo=2 PALU

def tiempo (data,k,metodo):
    alpha=0
    lista_1=[]
    lista_2=[]
    if data==1:
        dA = read_adjacency_matrix("adjacency1.dat")
    elif data==2:
        dA = read_adjacency_matrix("adjacency2.dat")
    elif data==3:
        dA = read_adjacency_matrix("adjacency3.dat")
    else:
        dA = read_adjacency_matrix("adjacency4.dat")
    for i in range(1,20):        
        tiempo_inicial = time() 
        if metodo==1:
            get_damping_ranking(dA, alpha, k, method='GMRes')
        else:
            get_damping_ranking(dA, alpha, k, method='PALU')

        lista_1.append(alpha)
        alpha=alpha+0.05
        tiempo_final = time() 
        tiempo_ejecucion = tiempo_final - tiempo_inicial
        lista_2.append(tiempo_ejecucion)
    plt.scatter(lista_2, lista_1)
    plt.title("alpha vs tiempo")
    plt.show()
    return 0

#tiempo(4,A,B,C,D,10,1)

interact(tiempo,data=IntSlider(min=1,max=4,step=1,value=3),k=IntSlider(min=1,max=100,step=1,value=10),metodo=(1,2,1))


interactive(children=(IntSlider(value=3, description='data', max=4, min=1), IntSlider(value=10, description='k…

<function __main__.tiempo(data, k, metodo)>

## Sección 3 : Análisis de iteraciones de GMRes

En esta sección debe analizar las soluciones obtenidas por GMRes en cada iteración, utilizando los datasets `web-NotreDame` y `web-Stanford`. Se recomienda modificar el código de GMRes de los Jupyter Notebook del curso, aunque no es obligatorio. **Importante:** Debido al tamaño de estos datasets, no se debe intentar cargar toda la matriz de adyacencia en memoria en formato denso

Considere la relación error $e_{k}$ versus iteración $k$, donde el error puede ser definido de la siguiente manera:

$$
e_{k} = \| \mathbf{x}_{k}-\mathbf{x}_{k-1} \|_2
$$

   Donde $\mathbf{x}_k$ es la solución de GMRes obtenida en la iteración $k$-ésima, con $k$ que **puede tomar valores** en el rango $[1, 2, \ldots, m]$. y $m$ el número de páginas del dataset. 
   
**1.** Utilice GMRes de manera conveniente para graficar el error $e_k$ versus $k$, utilizando un widget para variar el _damping factor_ $\alpha$ y seleccionar uno de los dos datasets requeridos. ¿Qué puede decir del error a medida que $k$ aumenta? ¿En qué afecta el valor de $\alpha$?

**Recomendación:** no intente cargar toda la matriz $\widehat{A}$ en memoria. En lugar de eso, considere que debido a que $\widehat{A}$ es _sparse_, la matriz $P$ también lo es y evite el cálculo de productos exteriores explícitamente. Se recomienda revisar el módulo `sparse` de `scipy` . Puede recurrir a modificaciones de GMRes para desarrollar esta pregunta. Utilice un valor máximo de $k$ razonable, pero no muy pequeño. No debe llegar necesariamente a $k = m$. Justifique su elección apropiadamente.

In [36]:
# import sparse module from SciPy package 
from scipy import sparse
# import uniform module to create random numbers
from scipy.stats import uniform

np.random.seed(seed=42)
data = uniform.rvs(size=1000000, loc = 0, scale=2)
data = np.reshape(data, (10000, 100))

data_size = data.nbytes/(1024**2)
print(data)
print('Size of full matrix with zeros: '+ '%3.2f' %data_size + ' MB')

data_csr = sparse.csr_matrix(data)
data_csr_size = data_csr.data.size/(1024**2)
print(data_csr)
print('Size of sparse csr_matrix: '+ '%3.2f' %data_csr_size + ' MB')

print("--------------------------------")
print(data_csr.todense())

[[0.74908024 1.90142861 1.46398788 ... 0.85508204 0.05083825 0.21578285]
 [0.06285837 1.27282082 0.62871196 ... 1.79422052 1.77417285 1.55975109]
 [1.28406329 0.16827993 0.32325743 ... 0.43164205 1.24578095 0.17069493]
 ...
 [1.18823974 1.01531142 0.41756942 ... 1.58206745 1.85359295 0.60241292]
 [0.16050398 1.97771288 1.15246621 ... 1.51226309 0.51651106 0.99064441]
 [0.58080629 1.43377084 1.99391229 ... 0.83614396 0.85734251 1.8588971 ]]
Size of full matrix with zeros: 7.63 MB
  (0, 0)	0.749080237694725
  (0, 1)	1.9014286128198323
  (0, 2)	1.4639878836228102
  (0, 3)	1.1973169683940732
  (0, 4)	0.31203728088487304
  (0, 5)	0.3119890406724053
  (0, 6)	0.11616722433639892
  (0, 7)	1.7323522915498704
  (0, 8)	1.2022300234864176
  (0, 9)	1.416145155592091
  (0, 10)	0.041168988591604894
  (0, 11)	1.9398197043239886
  (0, 12)	1.6648852816008435
  (0, 13)	0.4246782213565523
  (0, 14)	0.36364993441420124
  (0, 15)	0.36680901970686763
  (0, 16)	0.6084844859190754
  (0, 17)	1.0495128632644757


### Referencias

* Jupyter_Notebook_04_Metodos_Directos , del material de clases

* https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.gmres.html

* https://github.com/tclaudioe/Scientific-Computing/blob/master/SC1/10_GMRes.ipynb

* https://machinelearningmastery.com/sparse-matrices-for-machine-learning/